
# Projeto ICMC Júnior
## Objetivo: Prever se um funcionário vai sair da empresa (attrition).

### Etapa final: Modelagem dos dados

In [135]:
import numpy as np
import pandas as pd

In [136]:
# Leitura do dataset (usamos, agora, a base limpa gerada na etapa 1 do projeto)
df = pd.read_csv('dados_limpos.csv')

Realiza primeiro o mapeamento da coluna alvo e das colunas binárias (Label Encoding)

In [137]:
def encoding_binary_columns(column, map_values):
    
    if column not in df.columns:
        print(f"Erro, coluna {column} não encontrada no dataframe")
    
    print(f"Convertendo coluna: '{column}'...")
    print(f"  Valores ANTES: {df[column].unique()}")
    # Aplica mapeamento na coluna
    df[column] = df[column].map(map_values)
    print(f"Valores DEPOIS: {df[column].unique()}\n")
    

encoding_binary_columns('Attrition', {'yes': 1, 'no': 0})
encoding_binary_columns('OverTime', {'yes': 1, 'no': 0})
encoding_binary_columns('Gender', {'female': 1, 'male': 0})


Convertendo coluna: 'Attrition'...
  Valores ANTES: ['yes' 'no']
Valores DEPOIS: [1 0]

Convertendo coluna: 'OverTime'...
  Valores ANTES: ['yes' 'no']
Valores DEPOIS: [1 0]

Convertendo coluna: 'Gender'...
  Valores ANTES: ['female' 'male']
Valores DEPOIS: [1 0]



Converter as colunas categóricas nominais (One-Hot Encoding)

In [138]:
# Lista de colunas do tipo qualitativo nominal
colunas_nominais = [
    'BusinessTravel', 
    'Department', 
    'EducationField', 
    'JobRole', 
    'MaritalStatus'
]
        
# Realiza a conversão do dados de qualitativo para quantitativo, como consequência teremos um aumento de colunas de 31 para 45.
df = pd.get_dummies(df, 
                    columns=colunas_nominais,
                    dtype = int,
                    drop_first=True # Evita multicolinearidade
                    )

Com todas as colunas numéricas, agora podemos começar a modelagem dos dados aplicando o nosso modelo de Machile Learning